In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score
import time
import datetime
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [2]:
train_data = pd.read_csv('features.csv', index_col="match_id")
train_data

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,4,3,0,-24.0,2032,0,1792,1975,48,63
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,3,2,0,-17.0,1734,1,2038,6,63,3
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,1,3,1,-15.0,2906,0,1796,1846,51,63


In [3]:
##-- Исключаем ненужные признаки
train_data.drop(['start_time',
                    'duration',
                    'tower_status_radiant',
                    'tower_status_dire',
                    'barracks_status_radiant', 
                    'barracks_status_dire'
                ], axis=1, inplace=True)
train_data

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,0,42,4,1188,1033,9,0,1,12,49,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,7,33,4,1319,1270,22,0,0,12,98,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1,29,4,1779,1056,14,0,0,5,30,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,7,13,4,1431,1090,8,1,0,8,27,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1,47,4,1706,1198,17,0,1,8,7,...,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,0
114403,0,43,4,1793,1416,17,0,1,5,26,...,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0,1
114404,1,98,4,1399,540,1,0,0,5,11,...,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0,0


In [4]:
##-- Columns with empty cells
empty_rows = []
count_row = train_data.shape[0]
for column in train_data.columns:
    if train_data[column].count() != count_row:
        empty_rows.append(column)
        print(column)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [5]:
##--Fill empty rows with median value
for row in empty_rows:
    median = train_data[row].median()
    train_data[row].fillna(median, inplace=True)

In [6]:
train_data

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,0,42,4,1188,1033,9,0,1,12,49,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,7,33,4,1319,1270,22,0,0,12,98,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1,29,4,1779,1056,14,0,0,5,30,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,7,13,4,1431,1090,8,1,0,8,27,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1,47,4,1706,1198,17,0,1,8,7,...,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,0
114403,0,43,4,1793,1416,17,0,1,5,26,...,-5.0,131.0,-82.0,203.0,4,3,2,0,-17.0,1
114404,1,98,4,1399,540,1,0,0,5,11,...,-32.0,249.0,-70.0,203.0,1,1,3,1,-15.0,0


In [7]:
## Столбец с целевыми переменными
y = train_data['radiant_win']
train_data.drop('radiant_win', axis=1, inplace=True)
y_df = pd.DataFrame(y)
y_df

,radiant_win
match_id,
0,1
1,1
2,0
3,0
4,0
...,...
114402,0
114403,1
114404,0


In [8]:
##-- Признаки
X = train_data.copy()
X

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,0,42,4,1188,1033,9,0,1,12,49,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,7,33,4,1319,1270,22,0,0,12,98,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1,29,4,1779,1056,14,0,0,5,30,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,7,13,4,1431,1090,8,1,0,8,27,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1,47,4,1706,1198,17,0,1,8,7,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,0,43,4,1793,1416,17,0,1,5,26,...,0,-5.0,131.0,-82.0,203.0,4,3,2,0,-17.0
114404,1,98,4,1399,540,1,0,0,5,11,...,2,-32.0,249.0,-70.0,203.0,1,1,3,1,-15.0


In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

In [15]:
scores = defaultdict(list)
for n_estimators in (10, 20, 30):
    clf = GradientBoostingClassifier(n_estimators=n_estimators)
    for train_index, test_index in kf.split(X, y):
        start_time = datetime.datetime.now()
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf.fit(X_train, y_train)
        
        probs = clf.predict_proba(X_test)
        
        print('Time elapsed for n_estimaters={}: {}'.format(n_estimators, datetime.datetime.now() - start_time))
        
        scores[n_estimators].append(roc_auc_score(y_test, probs[:, 1]))
    scores[n_estimators] = np.mean(scores[n_estimators])

Time elapsed for n_estimaters=10: 0:00:08.611366
Time elapsed for n_estimaters=10: 0:00:08.513302
Time elapsed for n_estimaters=10: 0:00:08.369432
Time elapsed for n_estimaters=10: 0:00:08.485059
Time elapsed for n_estimaters=10: 0:00:08.509073
Time elapsed for n_estimaters=20: 0:00:16.667238
Time elapsed for n_estimaters=20: 0:00:16.578012
Time elapsed for n_estimaters=20: 0:00:16.483311
Time elapsed for n_estimaters=20: 0:00:16.280666
Time elapsed for n_estimaters=20: 0:00:16.508463
Time elapsed for n_estimaters=30: 0:00:24.816406
Time elapsed for n_estimaters=30: 0:00:25.007081
Time elapsed for n_estimaters=30: 0:00:24.780194
Time elapsed for n_estimaters=30: 0:00:24.758760
Time elapsed for n_estimaters=30: 0:00:24.832777


In [16]:
scores

defaultdict(list,
            {10: 0.6642534283486006,
             20: 0.6826487024110748,
             30: 0.6895551678562053})

In [18]:
##-- Второй этап задания
##
##
##
##

In [19]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
C = np.logspace(-3, 0, num=5)

In [20]:
def score_logistic_regression(X, y):
    best_score = 0
    best_c = 0
    for C in np.logspace(-3, 0, num=5):
        clf = LogisticRegression(penalty='l2', C=C)
        for train_index, test_index in kf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
            start_time = datetime.datetime.now()
            clf.fit(X_train, y_train)
            probs = clf.predict_proba(X_test)
            print('Time elapsed for C={}: {}'.format(C, datetime.datetime.now() - start_time))
        
            score = roc_auc_score(y_test, probs[:, 1])
            if score > best_score:
                best_score = score
                best_c = C
    return best_score, best_c

In [21]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:00.406912
Time elapsed for C=0.001: 0:00:00.440821
Time elapsed for C=0.001: 0:00:00.459798
Time elapsed for C=0.001: 0:00:00.461398
Time elapsed for C=0.001: 0:00:00.408980
Time elapsed for C=0.005623413251903491: 0:00:00.543079
Time elapsed for C=0.005623413251903491: 0:00:00.594467
Time elapsed for C=0.005623413251903491: 0:00:00.577472
Time elapsed for C=0.005623413251903491: 0:00:00.811266
Time elapsed for C=0.005623413251903491: 0:00:00.543063
Time elapsed for C=0.03162277660168379: 0:00:00.632034
Time elapsed for C=0.03162277660168379: 0:00:00.740542
Time elapsed for C=0.03162277660168379: 0:00:00.763549
Time elapsed for C=0.03162277660168379: 0:00:00.868793
Time elapsed for C=0.03162277660168379: 0:00:00.647301
Time elapsed for C=0.1778279410038923: 0:00:00.764137
Time elapsed for C=0.1778279410038923: 0:00:00.777475
Time elapsed for C=0.1778279410038923: 0:00:00.537590
Time elapsed for C=0.1778279410038923: 0:00:00.691712
Time elapsed for C=0.177

In [22]:
##-- Качество классификации над исходными признаками
best_score, best_c

(0.7187673078205024, 0.005623413251903491)

In [23]:
categorial_features = [
    'lobby_type',
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero'
]
X = train_data.drop(categorial_features, axis=1)
X

,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,5,2098,1489,20,0,0,7,3,842,991,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,4,1188,1033,9,0,1,12,4,1596,993,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,4,1319,1270,22,0,0,12,3,1314,775,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,4,1779,1056,14,0,0,5,2,539,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1431,1090,8,1,0,8,2,629,552,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,4,1706,1198,17,0,1,8,2,616,535,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,4,1793,1416,17,0,1,5,3,764,800,...,0,-5.0,131.0,-82.0,203.0,4,3,2,0,-17.0
114404,4,1399,540,1,0,0,5,4,1448,1371,...,2,-32.0,249.0,-70.0,203.0,1,1,3,1,-15.0


In [24]:
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,-0.332256,-0.625222,-0.255162,...,-0.987486,1.073562,-0.435542,-0.248595,0.313995,0.018054,0.562864,-0.551154,1.846004,-1.113822
1,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,0.578881,0.732454,-0.250795,...,-0.987486,-0.331318,0.370905,-0.248595,-0.562362,1.066668,0.562864,0.678170,0.437788,0.051388
2,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,-0.332256,0.224676,-0.726779,...,0.391203,-0.816640,-1.452366,0.211568,0.313995,0.018054,0.562864,0.678170,0.437788,0.497638
3,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-1.243393,-1.170813,-1.242065,...,-0.987486,-0.586750,-0.067381,0.014355,-0.933128,-1.554868,0.562864,-0.551154,-0.970428,0.844722
4,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-1.243393,-1.008757,-1.213680,...,-0.987486,1.354538,0.949443,0.014355,0.448819,1.590976,-0.302485,0.678170,-0.970428,-0.221321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,0.501314,0.834109,0.107949,0.637942,-0.537757,1.017574,-0.108632,-1.243393,-1.032165,-1.250798,...,-0.987486,-0.561207,0.914380,0.277305,-1.067952,0.018054,0.562864,0.678170,-0.970428,-0.419654
97226,0.501314,0.987660,0.577666,0.637942,-0.537757,1.017574,-1.309804,-0.332256,-0.765671,-0.672194,...,-0.987486,0.051831,0.055339,-0.117120,-0.292713,0.542361,-0.302485,-0.551154,-0.970428,-0.246112
97227,0.501314,0.292267,-1.309820,-1.131661,-0.537757,-0.578083,-1.309804,0.578881,0.465961,0.574534,...,1.769891,-0.637837,2.124050,0.671730,-0.292713,-1.030561,-2.033183,0.678170,0.437788,-0.196529
97228,-0.398181,-0.173682,-0.822866,-0.578660,-0.537757,2.613231,-0.909413,1.490017,1.377080,0.762308,...,-0.987486,-0.356861,2.211707,-0.314333,-0.966834,1.066668,-0.302485,0.678170,0.437788,-0.865905


In [25]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:00.491709
Time elapsed for C=0.001: 0:00:00.412899
Time elapsed for C=0.001: 0:00:00.526891
Time elapsed for C=0.001: 0:00:00.427138
Time elapsed for C=0.001: 0:00:00.386967
Time elapsed for C=0.005623413251903491: 0:00:00.576829
Time elapsed for C=0.005623413251903491: 0:00:00.589983
Time elapsed for C=0.005623413251903491: 0:00:00.524598
Time elapsed for C=0.005623413251903491: 0:00:00.565522
Time elapsed for C=0.005623413251903491: 0:00:00.585394
Time elapsed for C=0.03162277660168379: 0:00:00.631874
Time elapsed for C=0.03162277660168379: 0:00:00.607376
Time elapsed for C=0.03162277660168379: 0:00:00.605062
Time elapsed for C=0.03162277660168379: 0:00:00.680198
Time elapsed for C=0.03162277660168379: 0:00:00.615406
Time elapsed for C=0.1778279410038923: 0:00:00.680693
Time elapsed for C=0.1778279410038923: 0:00:00.668462
Time elapsed for C=0.1778279410038923: 0:00:00.692745
Time elapsed for C=0.1778279410038923: 0:00:00.657244
Time elapsed for C=0.177

In [26]:
##-- Качество без категориальных признаков
best_score, best_c

(0.7189620796414529, 0.005623413251903491)

In [27]:
##-- Всего героев в Dota2
r1_hero = np.array(train_data['r1_hero'])
r2_hero = np.array(train_data['r2_hero'])
r3_hero = np.array(train_data['r3_hero'])
r4_hero = np.array(train_data['r4_hero'])
r5_hero = np.array(train_data['r5_hero'])
d1_hero = np.array(train_data['d1_hero'])
d2_hero = np.array(train_data['d2_hero'])
d3_hero = np.array(train_data['d3_hero'])
d4_hero = np.array(train_data['d4_hero'])
d5_hero = np.array(train_data['d5_hero'])
heroes = np.array(set([*r1_hero, *r2_hero, *r3_hero, *r4_hero, *r5_hero,
                  *d1_hero, *d2_hero, *d3_hero, *d4_hero, *d5_hero]))
heroes

array({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 112},
      dtype=object)

In [29]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((train_data.shape[0], N))
print(X_pick)
for i, match_id in enumerate(train_data.index):
    for p in range(5):
        X_pick[i, train_data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [30]:
X = np.hstack((X, X_pick))
X_df1 = pd.DataFrame(X)
X_df1

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
0,5.0,2098.0,1489.0,20.0,0.0,0.0,7.0,3.0,842.0,991.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,1188.0,1033.0,9.0,0.0,1.0,12.0,4.0,1596.0,993.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,1319.0,1270.0,22.0,0.0,0.0,12.0,3.0,1314.0,775.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,1779.0,1056.0,14.0,0.0,0.0,5.0,2.0,539.0,539.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1431.0,1090.0,8.0,1.0,0.0,8.0,2.0,629.0,552.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,4.0,1706.0,1198.0,17.0,0.0,1.0,8.0,2.0,616.0,535.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97226,4.0,1793.0,1416.0,17.0,0.0,1.0,5.0,3.0,764.0,800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97227,4.0,1399.0,540.0,1.0,0.0,0.0,5.0,4.0,1448.0,1371.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97228,3.0,1135.0,766.0,6.0,0.0,2.0,6.0,5.0,1954.0,1457.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 1.40080818e+00,  1.52597175e+00,  7.34956866e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [ 5.01313538e-01, -8.01392943e-02, -2.47570363e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [ 5.01313538e-01,  1.51070097e-01,  2.63085236e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       ...,
       [ 5.01313538e-01,  2.92266672e-01, -1.30982020e+00, ...,
        -4.83865990e-03,  0.00000000e+00,  1.64609695e+00],
       [-3.98181106e-01, -1.73682025e-01, -8.22865912e-01, ...,
        -4.83865990e-03,  0.00000000e+00,  8.46061344e-04],
       [-3.98181106e-01, -3.18408515e-01, -7.51761967e-01, ...,
        -4.83865990e-03,  0.00000000e+00, -1.64440483e+00]])

In [32]:
best_score, best_c = score_logistic_regression(X_scaled, y)

Time elapsed for C=0.001: 0:00:00.754943
Time elapsed for C=0.001: 0:00:00.808628
Time elapsed for C=0.001: 0:00:00.662258
Time elapsed for C=0.001: 0:00:00.732413
Time elapsed for C=0.001: 0:00:00.754987
Time elapsed for C=0.005623413251903491: 0:00:00.959953
Time elapsed for C=0.005623413251903491: 0:00:00.927092
Time elapsed for C=0.005623413251903491: 0:00:01.110789
Time elapsed for C=0.005623413251903491: 0:00:01.000396
Time elapsed for C=0.005623413251903491: 0:00:01.091500
Time elapsed for C=0.03162277660168379: 0:00:01.208858
Time elapsed for C=0.03162277660168379: 0:00:01.228258
Time elapsed for C=0.03162277660168379: 0:00:01.391892
Time elapsed for C=0.03162277660168379: 0:00:01.269521
Time elapsed for C=0.03162277660168379: 0:00:01.333446
Time elapsed for C=0.1778279410038923: 0:00:01.265518
Time elapsed for C=0.1778279410038923: 0:00:01.438259
Time elapsed for C=0.1778279410038923: 0:00:01.323556
Time elapsed for C=0.1778279410038923: 0:00:01.272184
Time elapsed for C=0.177

In [33]:
##-- Качество при добавлении "мешка слов" по героям
best_score, best_c

(0.7567591978246206, 0.005623413251903491)

In [34]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test.drop([
    'start_time',
    'lobby_type'
], axis=1, inplace=True)
matches = X_test.shape[0]
X_test

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,93,4,1103,1089,8,0,1,9,102,3,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,20,2,556,570,1,0,0,9,6,4,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,112,2,751,808,1,0,0,13,26,2,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,27,3,708,903,1,1,1,11,91,2,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,39,4,1259,661,4,0,0,9,93,5,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,11,5,2054,1941,27,0,1,8,28,4,...,1,8.0,253.0,-87.0,NaN,4,3,2,1,-33.0
114377,3,3,748,605,1,0,0,12,22,3,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,85,2,575,499,0,0,0,8,102,3,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [35]:
##-- Columns with empty cells
empty_rows = []
count_row = X_test.shape[0]
for column in X_test.columns:
    if X_test[column].count() != count_row:
        empty_rows.append(column)
        print(column)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [36]:
##--Fill empty rows with median value
for row in empty_rows:
    median = X_test[row].median()
    X_test[row].fillna(median, inplace=True)

In [37]:
X_test

,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,93,4,1103,1089,8,0,1,9,102,3,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,20,2,556,570,1,0,0,9,6,4,...,2,-29.0,168.0,-54.0,203.0,3,2,2,1,16.0
10,112,2,751,808,1,0,0,13,26,2,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,27,3,708,903,1,1,1,11,91,2,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,39,4,1259,661,4,0,0,9,93,5,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,11,5,2054,1941,27,0,1,8,28,4,...,1,8.0,253.0,-87.0,203.0,4,3,2,1,-33.0
114377,3,3,748,605,1,0,0,12,22,3,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,85,2,575,499,0,0,0,8,102,3,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [38]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((X_test.shape[0], N))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0., -1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [39]:
X_test.drop([
    'r1_hero',
    'r2_hero',
    'r3_hero',
    'r4_hero',
    'r5_hero',
    'd1_hero',
    'd2_hero',
    'd3_hero',
    'd4_hero',
    'd5_hero'
], axis=1, inplace=True)

In [40]:
X_test

,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,4,1103,1089,8,0,1,9,3,1183,963,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,2,556,570,1,0,0,9,4,1194,1386,...,2,-29.0,168.0,-54.0,203.0,3,2,2,1,16.0
10,2,751,808,1,0,0,13,2,421,569,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,3,708,903,1,1,1,11,2,672,901,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,4,1259,661,4,0,0,9,5,1703,964,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,5,2054,1941,27,0,1,8,4,1460,936,...,1,8.0,253.0,-87.0,203.0,4,3,2,1,-33.0
114377,3,748,605,1,0,0,12,3,1130,1385,...,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,2,575,499,0,0,0,8,3,697,686,...,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [41]:
X_merged = np.hstack((X_test, X_pick))
X_merged_df1 = pd.DataFrame(X_merged)
X_merged_df1

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
0,4.0,1103.0,1089.0,8.0,0.0,1.0,9.0,3.0,1183.0,963.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,556.0,570.0,1.0,0.0,0.0,9.0,4.0,1194.0,1386.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,751.0,808.0,1.0,0.0,0.0,13.0,2.0,421.0,569.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.0,708.0,903.0,1.0,1.0,1.0,11.0,2.0,672.0,901.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
4,4.0,1259.0,661.0,4.0,0.0,0.0,9.0,5.0,1703.0,964.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,5.0,2054.0,1941.0,27.0,0.0,1.0,8.0,4.0,1460.0,936.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17173,3.0,748.0,605.0,1.0,0.0,0.0,12.0,3.0,1130.0,1385.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
17174,2.0,575.0,499.0,0.0,0.0,0.0,8.0,3.0,697.0,686.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
17175,4.0,1844.0,1176.0,8.0,1.0,2.0,8.0,3.0,1013.0,841.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [43]:
X_merged_scaled = scaler.fit_transform(X_merged)
X_merged_scaled_df1 = pd.DataFrame(X_merged_scaled)
X_merged_scaled_df1

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
0,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,-0.304443,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-0.002299
1,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,0.635111,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-0.002299
2,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,-1.179583,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,1.642924
3,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,-0.442155,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,-3.385453,0.0,-0.002299
4,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,-0.302222,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-0.002299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,1.394934,1.448970,1.699166,1.733795,-0.532732,1.003884,-0.093180,0.567766,0.505291,-0.364415,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-0.002299
17173,-0.445442,-0.867378,-1.180227,-1.145090,-0.532732,-0.593200,1.500200,-0.368985,-0.091940,0.632890,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-1.647522
17174,-1.365630,-1.174215,-1.408682,-1.255816,-0.532732,-0.593200,-0.093180,-0.368985,-0.875579,-0.919706,...,0.000682,0.004127,-0.012753,-2.282990,0.0,0.0,0.0,0.000789,0.0,-0.002299
17175,0.474746,1.076510,0.050412,-0.370005,0.968865,2.600967,-0.093180,-0.368985,-0.303685,-0.575426,...,0.000682,0.004127,-0.012753,0.006531,0.0,0.0,0.0,0.000789,0.0,-1.647522


In [44]:
clf = LogisticRegression(penalty='l2', C=1.0)

In [45]:
clf.fit(X_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
probs = clf.predict_proba(X_merged_scaled)[:, 1]

In [47]:
probs

array([0.8575132 , 0.76160057, 0.21645076, ..., 0.23000775, 0.59364837,
       0.42734649])

In [48]:
probs.shape

(17177,)

In [49]:
res = {'match_id': [], 'radiant_win': []}
for i, match_id in enumerate(X_test.index):
    res['match_id'].append(match_id)
    res['radiant_win'].append(probs[i])

In [52]:
res_df = pd.DataFrame(res)
res_df.set_index('match_id', inplace=True)

In [53]:
res_df

,radiant_win
match_id,
6,0.857513
7,0.761601
10,0.216451
13,0.861143
16,0.259881
...,...
114369,0.722603
114377,0.622806
114378,0.230008


In [54]:
res_df.to_csv('final-results.csv')

In [58]:
##-- Минимальное и максимальное значение прогноза на тестовой выборке
min(probs), max(probs)

(0.00888416176364562, 0.9966646436329507)